In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
#from utils import get_openai_api_key
#openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'wizardlm2:7b'
os.environ["OPENAI_API_BASE"]='http://localhost:11434/v1'
os.environ["OPENAI_API_KEY"]='NA'
os.environ["LOCAL_EMBEDDINGS"]="http://localhost:11434/api/embeddings"
os.environ["LOCAL_EMBEDDINGS_MODEL"]="nomic-embed-text:v1.5"

### Agent - Busca no Contexto

In [4]:
from search.context import utils
from crewai_tools import tool

# Embeddings Model
embeddings_model = "nomic-embed-text:v1.5"
# Vectore Store path
path_books = "./vector-store/chroma/books"

@tool("Pesquisar Livros")
def library_tool(query: str) -> str:
    """Retorna livros que possuem conteúdo relacionado com termo ou assunto buscado."""
    result = ""
    _, db = utils.retriver_context(embeddings_model=embeddings_model, path_books=path_books, vector_store='chroma')
    documents = db.similarity_search(query)
    for row in documents:
        page_content = row.page_content
        source = row.metadata['source']
        page = row.metadata['page']
        result += f"**{source.str.replace('./books/import/','')}** - {page}\n"
        result += f"{page_content}\n\n"
    return result

In [5]:
context_agent = Agent(
    role="Pesquisador acurado de livros",
	goal="Pesquisar livros com o conteúdo '{query}'.",
	backstory=(
		"Você é um pesquisador e bibliotecário especializado em livros."
        "Seu idioma é o português."
		"Assim você retornará citações completas para as perguntas ou "
        "assuntos do cliente e não faça suposições."
	),
	allow_delegation=False,
	verbose=True,
    tools=[library_tool],
)

mention_agent = Agent(
    role="Secretário especialista em citações",
	goal="Mencionar livros que possuem conteúdo relacionado com termo ou assunto buscado.",
	backstory=(
		"Você é um secretário especializado em citações de livros."
		"Seu idioma é o português."
		"Assim você retornará citações completas para as perguntas ou "
		"assuntos do cliente e não faça suposições."
	),
	allow_delegation=False,
	verbose=True
)

### Task - Busca no contexto

In [6]:
context_search = Task(
    description=(
        "1. Pesquise de livros para encontrar trechos "
            "relacionados ao tema {query}.\n"
        "2. A pesquisa deve ser feita no idioma nativo do cliente.\n"
    ),
    expected_output="Uma resposta que tenha um documento organizado "
        "com trechos de livros e suas fontes.",
    agent=context_agent,
)

In [7]:
mention_book = Task(
    description=(
        "1. Identifique os trechos de livros recebidos."
        "2. Avalie a relevância dos trechos de livros para o tema {query}.\n"
        "3. Selecione os trechos de livros mais relevantes e organize-os em um documento.\n"
    ),
    expected_output="Uma resposta que as citaçõesdos trechos de livros e suas fontes.",
    agent=mention_agent,
    context_agent=[context_search],
)

In [8]:
crew = Crew(
  agents=[context_agent, mention_agent],
  tasks=[context_search, mention_book],
  verbose=2,
  memory=False
)

In [9]:
inputs = {
    "query": "O que falta aos pregadores de hoje?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Pesquisador acurado de livros
 [INFO]: == Starting Task: 1. Pesquise de livros para encontrar trechos relacionados ao tema O que falta aos pregadores de hoje?.
2. A pesquisa deve ser feita no idioma nativo do cliente.



> Entering new CrewAgentExecutor chain...
 Action: Pesquisar Livros
Action Input: {"query": "O que falta aos pregadores de hoje"} 

**./books/import/Conselhos para obreiros - Charles H Spurgeon.pdf** - 11
“Pensei que ele não havia se preocupado muito conosco, porque sabia que, por onde quer que ele
passe, argumenta com seus anfitriões, e não havia feito assim conosco. Cheguei a pensar que
havíamos irritado o sr. Whitefield, mas agora percebo como ele foi amável conosco, falando-nos
dessa maneira.”
Em seguida chamou suas filhas: “Subam, meninas. Vejam o que o homem de Deus escreveu na
vidraça: ‘Falta-lhes uma coisa!’. Chamem seu pai”. O coronel subiu ao aposento de hóspedes e
também leu a frase: “Falta-lhes uma coisa!”. Assim, ao redor da cam